# TP1 du module 6 : les algorithmes de classification

Dans ce TP, nous allons mettre en pratique les principes de l'apprentissage supervisé. Objectifs :
* Savoir mettre en place les principaux algorithmes de classification
* Etudier l'impact de leurs paramètres sur leurs performances
* Comparer les performances de différents algorithmes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score,pairwise_distances
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

Commencez par charger à nouveau le jeu de données Titanic, à partir du csv généré dans le TP1 du module 4. Préparez les données d'entraînement et de test qui seront utilisées par la suite.

In [2]:
titanic = pd.read_csv('Titanic.csv')
titanic.head()

X = titanic.drop(['Survived'], axis=1)
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

## Partie 1 : découvrir Naive Bayes

1. Commencez par créer un modèle basé sur Naive Bayes, sans changer les paramètres par défaut, en supposant que la répartition des données correspond à une Gaussienne (loi normale). Entraînez-le et testez-le. Quelle score (accuracy) obtenez-vous ? Que pouvez-vous dire de la précision et du rappel ? Comparez avec les scores obtenus sur les arbres de décision au module 5 : avez-vous des hypothèses pour expliquer cette différence ?

In [3]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

print("Accuracy :", accuracy_score(y_test, y_pred))
print("Rappel :", recall_score(y_test, y_pred))
print("Pecision :", precision_score(y_test, y_pred))

***Observation***: l'accuracy est faible

2. Affichez une matrice de corrélation des données du jeu d'entraînement, en y incluant un affichage textuel de la valeur de la corrélation. Voyez-vous des informations permettant d'expliquer les performance de l'algorithme Naive Bayes ?

In [4]:
sns.heatmap(titanic.corr(), annot=True)

3. Proposez une représentation graphique des attributs continus, permettant de vérifier l'hypothèse que nous avons faite, selon laquelle ces données suivent une loi normale (Gaussienne).

In [5]:
fig, axes= plt.subplots(1,2, figsize=(20,5))
index=0
for attribut in ['Age', 'Fare']:
    sns.histplot(data=X, x=attribut, hue='Fare', kde=True, ax=axes[index])
    index+=1

## Partie 2 : découvrir KNN

1. Commencez par créer un modèle knn, en gardant le nombre de voisins par défaut (à regarder dans la documentation). Que pouvez-vous dire de l'accuracy, de la précision et du rappel ?

In [6]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("Accuracy :", accuracy_score(y_test, y_pred))

2. Nous allons maintenant observer l'impact du nombre de voisins à prendre en considération. Faite varier k entre 1 et 20. Calculez à chaque fois accuracy, précision, et rappel. Tracez l'évolution de ces trois scores en fonction de k, sur un même graphique. Que constatez-vous ? Affichez la valeur de k pour laquelle l'accuracy est la plus élevée.

In [7]:
accuracy = []
rappel = []
precision = []

for k in range(1,21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    
plt.plot(range(1,21),accuracy, marker='o', label='Accuracy', color='b')
plt.plot(range(1,21),rappel, marker='o', label='Recall', color='g')
plt.plot(range(1,21),precision, marker='o', label='Precision', color='r')
plt.xticks(range(1, 21))
plt.xlabel("P")
plt.legend()
plt.show()
print(rappel)

***Observetion*** : L'accuracy estla plus eleve pour le k=19

3. En prenant la valeur de k qui vous semble la plus pertinente, faite varier la dimension (p) utilisée pour calculer la distance de Minkowski entre deux données. Cette distance a-t'elle un fort impact sur les résultats d'accuracy obtenus ? Montrez-le en montrant l'évolution de ce score en fonction de p (faire varier entre 1 et 10). Ajoutez également la précision et le rappel.

In [8]:
accuracy = []
rappel = []
precision = []
for p in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=3, p=p)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))

plt.plot(range(1,11),accuracy, marker='o', label='Accuracy', color='b')
plt.plot(range(1,11),rappel, marker='o', label='Recall', color='g')
plt.plot(range(1,11),precision, marker='o', label='Precision', color='r')
plt.xticks(range(1, 11))
plt.xlabel("P")
plt.legend()
plt.show()
print(accuracy)

## Partie 3 : découvrir les SVM

1. Créez un modèle de classification basée sur les machines à vecteur de support. Dans un premier temps, gardez les options par défaut. Que pouvez-vous dire des performances obtenues (accuracy, précision, rappel) ?

In [9]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test) 
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Rappel :", recall_score(y_test, y_pred))
print("Precision :", precision_score(y_test, y_pred))

2. Testez les différents noyaux disponibles pour l'algorithme SVM (linéaire, polynomial, rbf et sigmoïde). Représentez graphiquement l'accuracy, la précision et le rappel, pour chaque noyau. Il y en a t'il un qui semble plus pertinent que les autres ? Affichez-le, ainsi que les scores obtenus pour ce noyau.

In [10]:
kernels = ["linear","poly","rbf","sigmoid"]
accuracy = []
rappel = []
precision = []

for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test) 
    accuracy.append(accuracy_score(y_test, y_pred))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))


print(accuracy)
print(rappel)
print(precision)
plt.plot(kernels, accuracy, marker='o', label='Accuracy', color='b')
plt.plot(kernels, rappel, marker='o', label='Recall', color='g')
plt.plot(kernels, precision, marker='o', label='Precision', color='r')
plt.xlabel("P")
plt.legend()
plt.show()

3. Nous allons essayer d'améliorer les performances obtenues avec le noyau polynomial. Utilisez ce noyau, et faites varier le degré du polynôme utilisé de 1 à 10. Représentez graphiquement l'accuracy, la précision et le rappel, en fonction du degré du polynôme. Il y en a t'il un qui semble plus pertinent que les autres ? Affichez-le, ainsi que les scores obtenus pour cette valeur. Comparez avec le meilleur score obtenu à la question précédente.

In [11]:
accuracy = []
rappel = []
precision = []

for d in range(1,11):
    svm = SVC(kernel='poly', degree=d)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test) 
    accuracy.append(svm.score(X_test, y_test))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))


print(accuracy)
print(rappel)
print(precision)
plt.plot(range(1,11), accuracy, marker='o', label='Accuracy', color='b')
plt.plot(range(1,11), rappel, marker='o', label='Recall', color='g')
plt.plot(range(1,11), precision, marker='o', label='Precision', color='r')
plt.xlabel("D")
plt.xticks(range(1, 11))
plt.legend()
plt.show()

## Partie 4 : découvrir les réseaux de neurones

1. Commençons par étudier le réseau le plus simple : un perceptron. A l'aide de la classe `sklearn.linear_model.Perceptron`, créez un perceptron, en gardant les options par défaut. Affichez accuracy, précision et rappel : que pensez-vous de ces performances ?

In [12]:
percp = Perceptron()
percp.fit(X_train, y_train)
y_pred = percp.predict(X_test) 
print("Accuracy :", percp.score(X_test, y_test))
print("Rappel :", recall_score(y_test, y_pred))
print("Precision :", precision_score(y_test, y_pred))

2. Regardez la documentation pour créer un réseau de neurones (`sklearn.neural_network.MLPClassifier`) : quelle est la structure d'un réseau de neurones par défaut avec scikit-learn ? Combien de couches cachées ? Combien de neurones par couche ?

***Nombre de couche***: 1
***Nombre de neuronne***: 100

2. Créer un réseau de neurones, en gardant ces options par défaut. Affichez accuracy, précision et rappel : que pensez-vous de ces performances, notamment en comparant par rapport au perceptron ? Avez-vous un message d'alerte ?

In [13]:
ann = MLPClassifier()
ann.fit(X_train, y_train)
y_pred = ann.predict(X_test) 
print("Accuracy :", ann.score(X_test, y_test))
print("Rappel :", recall_score(y_test, y_pred))
print("Precision :", precision_score(y_test, y_pred))

***Observation*** : l'accuracy et la precision sont plus eleve avec le reseau de neuronnes qu'avec le percpttron, mais le rapelle lui a diminuer

3. Si vous avez observé un message d'alerte sur la question précédent, que signifie-t'il selon vous ? Que pouvez-vous faire pour y remédier ? Proposez un code permettant d'obtenir des résultats, sans message d'alerte. Qu'observez-vous sur l'évolution des scores ?

In [14]:
ann = MLPClassifier(max_iter=1000)
ann.fit(X_train, y_train)
y_pred = ann.predict(X_test) 
print("Accuracy :", ann.score(X_test, y_test))
print("Rappel :", recall_score(y_test, y_pred))
print("Precision :", precision_score(y_test, y_pred))

4. Nous allons à présent comparer différentes architectures du réseau de neurones :
- Trois couches de 50 neurones chacune
- Cinq couches de 50 neurones chacune
- Trois couches : première avec 50, deuxième avec 100, troisième avec 50 neurones
- Cinq couches : première avec 50, deuxième avec 100, troisième avec 50 neurones, quatrième avec 100, cinquième avec 50 neurones

Représentez graphiquement l'accuracy, la précision et le rappel, pour chaque architecture. Il y en a t'il une qui semble plus pertinente que les autres ? Affichez-la, ainsi que les scores obtenus pour cette architecture. Comparez avec le score obtenu par l'architecture par défaut. Votre code ne doit générer aucun message d'alerte.

In [15]:
accuracy = []
rappel = []
precision = []
architectures = [
    (50,50,50),
    (50,50,50,50,50),
    (50,100,50),
    (50,100,50,100,50)
]
for arch in architectures:
    ann = MLPClassifier(hidden_layer_sizes=arch, max_iter=1000)
    ann.fit(X_train, y_train)
    y_pred = ann.predict(X_test) 
    accuracy.append(svm.score(X_test, y_test))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))

x = np.arange(len(architectures))

print(accuracy)
print(rappel)
print(precision)
plt.plot(x, accuracy, marker='o', label='Accuracy', color='b')
plt.plot(x, rappel, marker='o', label='Recall', color='g')
plt.plot(x, precision, marker='o', label='Precision', color='r')
plt.xlabel("Architecture")
plt.xticks(x, architectures)
plt.legend()
plt.show()

5. En utilisant l'architecture qui vous donnait les meilleures performances, étudier l'impact de la fonction d'activation utilisée sur les performances. Représentez sur un graphiques les scores (accuracy, précision et rappel) obtenus pour les quatres fonctions d'activation proposées par scikit-learn. Affichez la fonction qui vous parait la plus pertinente, ainsi que les scores associés.

In [16]:
activations=['identity', 'logistic', 'tanh', 'relu']
accuracy = []
rappel = []
precision = []
for activ in activations: 
    ann = MLPClassifier(hidden_layer_sizes=(50,100,50,100,50), activation=activ)
    ann.fit(X_train, y_train)
    y_pred = ann.predict(X_test) 
    accuracy.append(svm.score(X_test, y_test))
    rappel.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))

print(accuracy)
print(rappel)
print(precision)
plt.plot(activations, accuracy, marker='o', label='Accuracy', color='b')
plt.plot(activations, rappel, marker='o', label='Recall', color='g')
plt.plot(activations, precision, marker='o', label='Precision', color='r')
plt.xlabel("Activations")
plt.xticks(activations)
plt.legend()
plt.show()

## Partie 5 : comparer les performances des différents algorithmes

Nous allons à présent résumer les différentes performances des algorithmes que vous avez testé dans ce TP : récupérez les meilleurs scores (accuracy) obtenu pour chaque algorithme. Représentez-les sur un diagramme en barres, en regroupant par algorithme, et en représentant chaque score par une couleur. Un algorithme semble-t'il obtenir de meilleures performances que les autres ?

## Partie 6 : optimiser la recherche des paramètres optimaux

Dans ce TP, nous avons souvent cherché à identifier la meilleur combinaison de paramètres. Nous avons procédé par itération, en cherchant à fixer un paramètre avant de faire évoluer les autres. Cette méthode est couteuse, et pour faire une recherche exhaustive, nécessite, de répéter très souvent le même code. Scikit-learn propose une classe, `sklearn.model_selection.GridSearchCV`, qui va permettre d'optimiser cette recherche de paramétrage optimal.

Lien vers la documentation : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Le principe est de définir un dictionnaire, où la clé correspond à un paramètre, et la valeur à la liste de valeurs possibles à tester pour le paramètre considéré. 

**Consigne :** Appliquez ce principe pour déterminer la meilleure combinaison possible pour le réseau de neurones, en repartant des différentes configurations testées dans les parties précédentes.

In [17]:
parameters = {
    'kernel' : ()
}